<a href="https://colab.research.google.com/github/Bunsekiya/intern21/blob/master/%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%81%AB%E3%82%88%E3%82%8B%E8%A1%A3%E6%96%99%E5%93%81%E3%81%AE%E5%88%86%E9%A1%9E200304.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ニューラルネットワークによる衣料品の分類

スニーカーやシャツなど、身に着けるものの写真を分類するニューラルネットワークのモデルを作成します。

TensorFlowのモデルを構築し訓練するためのAPIは [tf.keras](https://www.tensorflow.org/guide/keras)を使用します。

In [0]:
#　使用するtensorflowのバージョンを指定する
%tensorflow_version 2.x

from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow と tf.keras のインポート
import tensorflow as tf

from tensorflow import keras

# ヘルパーライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## ファッションMNISTデータセットのロード

データセットは、[Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist)を使用します。Fashion MNISTには10カテゴリーの白黒画像70,000枚が含まれています。それぞれは下図のような1枚に付き1種類の衣料品が写っている低解像度（28×28ピクセル）の画像です。

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

ここでは、60,000枚の画像を訓練に、10,000枚の画像を、ネットワークが学習した画像分類の正確性を評価するのに使います。
以下のようにTensorFlowを使ってFashion MNISTのデータをインポートし、ロードすることが出来ます。

In [0]:
# Fashion Mnistの取り込み（インポート）
fashion_mnist = keras.datasets.fashion_mnist

# 画像データをtrain*（画像、ラベル）とtest*(画像、ラベル)に分ける
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

* train_images と train_labels の2つの配列は、モデルの訓練に使用される訓練用データセットです。
* 訓練されたモデルは、 test_images と test_labels 配列からなるテスト用データセットを使ってテストします。

画像は28×28ピクセルで構成されています。それぞれのピクセルの値は0から255の間の整数です。ラベル（label）は、0から9までの整数の配列です。それぞれの数字が下表のように、衣料品のクラス（class）に対応しています。

<table>
  <tr>
    <th>Label</th>
    <th>Class</th> 
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td> 
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td> 
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td> 
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td> 
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td> 
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td> 
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td> 
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td> 
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td> 
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td> 
  </tr>
</table>

## データの観察

モデルの訓練を行う前に、データセットのフォーマットを見てみましょう。下記のように、訓練用データセットには28×28ピクセルの画像が60,000枚含まれています。

In [0]:
# トレーニングをする画像の数と縦幅横幅の大きさを表示
print(train_images.shape)

同様に、訓練用データセットには60,000個のラベルが含まれます。

In [0]:
# トレーニングをするラベルの数を表示
print(len(train_labels))

ラベルはそれぞれ、0から9までの間の整数です。

In [0]:
# ラベルを表示
print(train_labels)

テスト用データセットには、10,000枚の画像が含まれます。画像は28×28ピクセルで構成されています。

In [0]:
# テストをする画像の数と縦幅横幅の大きさ
print(test_images.shape)

テスト用データセットには10,000個のラベルが含まれます。

In [0]:
# テストをするラベルの数
print(len(test_labels))

## データの前処理

ネットワークを訓練する前に、データを前処理する必要があります。
以下のコードを実行し、画像の幅が28✕28になっていること、ピクセルの値は0から255の間の数値になっていることを確認しましょう。

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.gca().grid(True)

**訓練用データセット**の最初の10枚の画像を、クラス名付きで表示してみましょう。

In [0]:
import matplotlib.cm as cm

# このまま学習させると処理に大きな負荷を掛けるため学習データをスケーリング（0から1の範囲）していきます。
train_images = train_images / 255.0
test_images = test_images / 255.0

# 服ラベル（0～9）のクラス分けを行うための準備をします。
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# 図を描画する
plt.figure(figsize=(10,10)) # 大きさを指定
for i in range(25):
    plt.subplot(5,5,i+1) # 配置指定
    plt.xticks([]) # x軸の目盛りを取得または設定
    plt.yticks([]) # y軸の目盛りを取得または設定
    plt.grid(False) # グリッド線を非表示に設定
    plt.imshow(train_images[i], cmap=plt.cm.binary) # 出力する色合いを指定
    plt.xlabel(class_names[train_labels[i]]) # x軸のラベルを出力

## モデルの構築

ニューラルネットワークを構築するには、まずモデルの階層を定義し、その後モデルをコンパイルします。

### 層の設定

* 入力層

このネットワークの最初の層は、画像を（28×28ピクセルの）2次元配列から、28×28＝784ピクセルの、1次元配列に変換します。

* 中間層

ニューロンの活性化によって判別に有利な特徴を抽出します。中間層では**層の数**や**ニューロン数**の設定が特徴の抽出に影響します。また、中間層の処理につかわれる**活性化関数**には**linear、hard_sigmoid、sigmoid、tanh、relu、softsign、softplus**などがあります。

* 出力層

最後の層は、10ニューロンのsoftmax層です。それぞれのニューロンは、画像が10個のクラスのひとつひとつに属する確率を出力します。

### モデルのコンパイル

* **オプティマイザ**（optimizer）勾配降下法によって出力ニューロンに対する重みを最適化します。現在公開されているオプティマイザには**SGD、RMSprop、Adagrad、Adadelta、Adam、Adamax、Nadam**などがあります。

In [0]:
#層の設定
model = keras.Sequential([
    # 入力を2次元から1次元に落とす
    keras.layers.Flatten(input_shape=(28, 28)), #1層目

    # 中間層をノード数○〇を設定し、活性化関数を✕✕に設定
    keras.layers.Dense(512, activation='relu'), #2層目
    
    # 最後の出力は10段階で、それぞれの確率がでるようにする。
    keras.layers.Dense(10, activation='softmax') #3層目
])

#モデルのコンパイル(プログラミング言語をPC語に変換)
model.compile(optimizer='RMSprop', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy','sparse_categorical_crossentropy'])

## モデルの訓練

ニューラルネットワークの訓練にはmodel.fit メソッドを呼び出します。モデルを訓練用データに "fit"（適合）させるという意味です。学習の試行回数（**エポック数**）はepochsで指定します。訓練は以下のステップで行われています。

1. モデルに訓練用データを投入します—この例では train_images と train_labels の２つの配列です。
2. モデルは、画像とラベルの対応関係を学習します。
3. モデルにテスト用データセットの予測（分類）を行わせます—この例では test_images 配列です。その後、予測結果と test_labels 配列を照合します。 

In [0]:
# epochs 訓練を何回繰り返すか
model_history = model.fit(train_images, train_labels, epochs=50, batch_size=512, validation_data=(test_images, test_labels), verbose=2)

モデルの訓練の進行とともに、損失値と正解率が表示されます。

## 正解率の評価

次に、テスト用データセットに対するモデルの性能を比較します。

In [0]:
# test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
# print('\nTest accuracy:', test_acc)

# エポック毎のモデル性能をグラフに出力します
def plot_history(histories, key):
  plt.figure(figsize=(8,5))
    
  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([0,max(history.epoch)])

plot_history([('baseline', model_history)], key='sparse_categorical_crossentropy')
plot_history([('baseline', model_history)], key='accuracy')

ご覧の通り、テスト用データセットでの正解率は、訓練用データセットでの正解率よりも少し低くなります。この訓練時の正解率とテスト時の正解率の差は、過学習（over fitting）の一例です。過学習とは、新しいデータに対する機械学習モデルの性能が、訓練時と比較して低下する現象です。

## 予測する

モデルの訓練が終わったら、そのモデルを使って画像の分類予測を行うことが出来ます。

In [0]:
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'green'
    else:
        color = 'yellow'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                    100*np.max(predictions_array),
                                    class_names[true_label]),
                                    color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1]) 
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('yellow')
    thisplot[true_label].set_color('green')

predictions = model.predict(test_images)

# print(np.argmax(predictions[0]))

i=2 #確認したいデータNo.を指定する
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.figure(figsize=(12,3))
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)
plt.show()

モデルがテスト用データセットの画像のひとつひとつを分類予測した結果です。これは、その画像が10の衣料品の種類のそれぞれに該当するかの「確信度」を表しています。

この確信度が正しいかどうか、テスト用ラベルを見てみましょう。

In [0]:
print(test_labels[2])

## 課題

モデルのハイパーパラメータを調整してモデルの正解率を向上させよ。

**ヒント：層の数、ニューロン数、活性化関数、オプティマイザ、エポック数**